<a href="https://colab.research.google.com/github/WoradeeKongthong/medical_cost_regression/blob/master/05_Medical_Cost_SVR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
## Data Preprocessing

# importing the dataset
df = pd.read_csv('/content/drive/My Drive/life-long learning/MyProjects/Medical Cost/cleaned_insurance.csv', usecols=[0,1,2,3,4,5,6])

In [20]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,True,southwest,16884.92400
1,18,male,33.770,1,False,southeast,1725.55230
2,28,male,33.000,3,False,southeast,4449.46200
3,33,male,22.705,0,False,northwest,21984.47061
4,32,male,28.880,0,False,northwest,3866.85520


In [0]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [22]:
print(X)

[[19 'female' 27.9 0 True 'southwest']
 [18 'male' 33.77 1 False 'southeast']
 [28 'male' 33.0 3 False 'southeast']
 ...
 [18 'female' 36.85 0 False 'southeast']
 [21 'female' 25.8 0 False 'southwest']
 [61 'female' 29.07 0 True 'northwest']]


In [23]:
print(y)

[16884.92  1725.55  4449.46 ...  1629.83  2007.94 29141.36]


In [0]:
y = y.reshape(len(y),1)

In [0]:
# encoding categorical data
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(drop='first'), [1,4,5])], remainder='passthrough')
X = ct.fit_transform(X)

In [25]:
print(X[:5])

[[0.0 1.0 0.0 0.0 1.0 19 27.9 0]
 [1.0 0.0 0.0 1.0 0.0 18 33.77 1]
 [1.0 0.0 0.0 1.0 0.0 28 33.0 3]
 [1.0 0.0 1.0 0.0 0.0 33 22.705 0]
 [1.0 0.0 1.0 0.0 0.0 32 28.88 0]]


In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
sc_y = StandardScaler()

X_train[:,5:] = sc_X.fit_transform(X_train[:,5:])
X_test[:,5:] = sc_X.transform(X_test[:,5:])

y_train = sc_y.fit_transform(y_train)

In [36]:
# Training the SVR model on the Training set
from sklearn.svm import SVR

#Best Parameters :  {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}

regressor = SVR(kernel = 'rbf', gamma='auto',C=10)
regressor.fit(X_train, y_train)

SVR(C=10, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [37]:
# Predicting the Test set results
y_pred = sc_y.inverse_transform(regressor.predict(X_test))
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)[:10])

[[14252.21 14283.46]
 [14398.69 14571.89]
 [17606.23 16232.85]
 [ 4042.61  2904.09]
 [ 2459.58  2480.98]
 [12222.28 11840.78]
 [ 9878.87  2438.06]
 [ 4217.48  2803.7 ]
 [ 8355.88  9058.73]
 [ 9933.44 13747.87]]


In [42]:
# Model Evaluation
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt
n = X_test.shape[0]
p = X_test.shape[1]
r_square = r2_score(y_test, y_pred)
adj_r_square = 1 - (1 - r_square) * ((n - 1) / (n - p - 1)) 
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mean_squared_error(y_test, y_pred))

print("R-square : ",r_square)
print("Adjusted R-square : ",adj_r_square)
print("MAE : ", mae)
print("MSE : ",mse)
print("RMSE : ",rmse)

R-square :  0.842725923405255
Adjusted R-square :  0.83786803686951
MAE :  2345.007021774045
MSE :  20472350.12143189
RMSE :  4524.638120494487


In [39]:
# Applying K-Fold Cross Validation
from sklearn.model_selection import cross_val_score
y_train = y_train.reshape(len(y_train),)
accuracies = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10)
print("Accuracy: {}".format(accuracies.mean()))
print("Standard Deviation: {}".format(accuracies.std()))

Accuracy: 0.8407260814993069
Standard Deviation: 0.04875161268611396


In [35]:
# Improving the model

# Grid Search
from sklearn.model_selection import GridSearchCV
parameters = [{'C':[1,10,100],'kernel':['linear']},
              {'C':[1,10,100],'kernel':['rbf'],'gamma': ['scale','auto']},]
grid_search = GridSearchCV(estimator = regressor,
                          param_grid = parameters,
                          cv = 10)
grid_search = grid_search.fit(X_train,y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy : {}".format(best_accuracy))
print("Best Parameters : ", best_parameters)

Best Accuracy : 0.8407260814993069
Best Parameters :  {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}


In [43]:
#-----------------training evaluation--------------------------------
n = X_train.shape[0]
p = X_train.shape[1]
r_square = r2_score(y_train, regressor.predict(X_train))
adj_r_square = 1 - (1 - r_square) * ((n - 1) / (n - p - 1)) 
mae = mean_absolute_error(y_train,  regressor.predict(X_train))
mse = mean_squared_error(y_train, regressor.predict(X_train))
rmse = sqrt(mean_squared_error(y_train,  regressor.predict(X_train)))

print("R-square : ",r_square)
print("Adjusted R-square : ",adj_r_square)
print("MAE : ", mae)
print("MSE : ",mse)
print("RMSE : ",rmse)

R-square :  0.8624554581585205
Adjusted R-square :  0.8614183645348336
MAE :  0.18172149138064184
MSE :  0.1375445418414795
RMSE :  0.37086997969838364
